# Graph of words

Given a sequence of words $s_1,...s_m$, we perform a $n-gram$

# 1. Data processing 
w.l.o.g. we use a corpora using all words in the input txt.

# 2. Models

## 2.1. Universal isomorphic embedding (word as graph node)



In [181]:
import collections
from operator import itemgetter
import re
import nltk
import numpy as np
from nltk.corpus import stopwords                                  
from itertools import permutations
import matplotlib.pyplot as plt
import word2vec
import pandas as pd
from numpy import save
import networkx as nx
from sklearn.decomposition import PCA
import gensim
from nltk import word_tokenize
from collections import Counter
import nltk
import copy
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import string

# Load Google's pre-trained Word2Vec model.
WORD2VEC_VEC = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


### 2.1.1. Preprocessing 

In [ ]:
def tokenize(text):
    """ Will split a text into words and remove stop words
    param:
         text: string - can be a document or a query

    return: 
         a list of words
    """
    words = None
    # the text into a sequence of words and store it in words
    words = text.split()
    
    # # remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    
    
    return words
def myindex(cleaned_texts):
    """ use a dictionary (HashMap) for storing the inverted index
    param:
        cleaned_texts:  

    return: 
        inverted_index: key=word, value=word index in vertex list 
        vertex_list: list of word index
        split_texts: a list of words 
    """
    split_texts = tokenize(cleaned_texts) 
    inverted_index = {}
    split_text_set = set(split_texts)
    vertex_list = []
    for index, word in enumerate(split_text_set):
            inverted_index[word] = inverted_index.get(word, [])
            inverted_index[word].append(index)
            vertex_list.append(word)
    return inverted_index, vertex_list, split_texts

def generate_edge(word_list, n):
    """ use a sliding window of size n, count the number of pairs as edge weights
        input size: n
    param:
        decoded_texts: a list of vertex index
        n: sliding window length

    return: 
        edge_mat: L x L matrix, L being size of word set
    """
    NUM_DISTINCT_WORD = max(word_list) + 1
    edge_mat = np.zeros([NUM_DISTINCT_WORD, NUM_DISTINCT_WORD])
    pair_list = []

    for i in range(len(word_list)-n+1):
        pair_list.append(list(permutations(word_list[i:i+n],2))) 

    for i in range(len(pair_list)):
        for it in pair_list[i]:
            edge_mat[it[0],it[1]] += 1
    return edge_mat 



def retrive_inverted_index(txt_file_name = 'test800.txt'):
    """ input .txt file name and return inverted index 
    param:
        txt_file_name: txt file name to read

    return: 
        inverted_index: a dictionary (HashMap) for storing the inverted index
    """

	text_content = open(txt_file_name, encoding = 'utf-8', errors = 'ignore').read()
	# preprocess 
	cleaned_texts = []
	regex = re.compile('[^a-zA-Z]') # filter only letters
	for innerText in text_content:
	    cleaned_texts.append(regex.sub(' ', innerText))  

	# convert into lowercase
	for index in range(0, len(cleaned_texts)):
	    cleaned_texts[index] = cleaned_texts[index].lower()
	cleaned_texts = ''.join(cleaned_texts) # convert back to str
	# print('cleaned length',len(cleaned_texts))


	inverted_index, vertex_list, split_texts = myindex(cleaned_texts)
	# encode the whole txt (tokenized) into its vertex number 
	decoded_texts = [inverted_index[word][0] for word in split_texts] # contain vertex index 
	# print('decoded length',len(decoded_texts),len(split_texts))
	# print('word set length', len(set(inverted_index)))# generate edge weight matrix 
	edge_mat_dict = {}
	# print('decoded_texts length:',len(decoded_texts))

    # ####### use sentence as vertex #######
    # sentences = sent_tokenize(text_content)
    # sentences = clean_sentences(sentences)
    # graph_vertices, inverted_index_sentence = get_vertices(sentences)



	return inverted_index



### 2.1.2. Generate train and test data

In [214]:
# map 26 tags to 5 self-defiend tages 
tag_dict = {'CD':4, 'CC':4, 'DT':4, 'EX':4, 'FW':4, 'IN':4, 'LS':4, 'MD':4, 'WP':4,'WP$':4,'PRP':4,'TO':4, 'UH':4,'WDT':4,'WRB':4,
            'JJ':0,'JJS':0,'JJR':0,
            'NN':1,'NNS':1,'NNP':1,'NNPS':1,
            'RB':2, 'RBR':2, 'RP':2,
            'VB':3, 'VBD':3, 'VBG':3, 'VBN':3, 'VBP':3, 'VBZ':3}
label_num_to_tag_dict = {0: 'Adjective', 
                    1: 'Noun',
                    2: 'Adverb',
                    3: 'Verb',
                    4: 'Others'}

def retrive_X_Y_all(uie_dat_name, txt_file_name, k):
    """ retrieve data and labels 
	Input:	
		uie_dat_name: file name of the .npy file
		inverted_index: orginal inevrted index
		k: reduced dimension 

	Return:
		X_gv: retrived from google vector  
		X_uie: retrived from universal isotropic embedding 
		Y: labels (pos tagging)
	"""
    inverted_index = retrive_inverted_index(txt_file_name)
    

    ########## Google vector ##########
    # words google vectors do not contain 
    error_words = []
    for ele in inverted_index.keys():
        try:
            WORD2VEC_VEC[ele]
            # raise ValueError("I have raised an Exception")
        except KeyError as exp:
            error_words.append(ele)

    # new_inverted_index only has what google vector contaion 
    new_inverted_index = copy.deepcopy(inverted_index) 
    for ele in error_words:
        del new_inverted_index[ele] 

    unique_texts = list(new_inverted_index.keys())

    vec = [WORD2VEC_VEC[ele] for ele in unique_texts]
    pca = PCA(n_components=k)
    X_wtv = pca.fit_transform(vec) # google vector 
    print('PCA total explained ratio:', sum(pca.explained_variance_ratio_))


    ########## universal isotropic embedding  ##########
    # load data 
    vec_uie = np.load(uie_dat_name)
    # remove col and rows of words not appearing in google vector 
    temp = np.array(vec_uie)
    for word in error_words:
        idx = inverted_index[word]
        temp = np.delete(temp, idx, axis=0)
        temp = np.delete(temp, idx, axis=1)
    X_uie = pca.fit_transform(temp) # universal isotropic embedding  
    
    ########## Retrive labels  ##########
    pos_tags0 = nltk.pos_tag(list(new_inverted_index.keys())) # get the pos tag for each vertex
    print('original occurance: ',Counter(elem[1] for elem in pos_tags0))
    set(ele[1] for ele in pos_tags0)

    # check if all predicted label by nltk in tag_dict 
    for ele in pos_tags0:
        if ele[1] not in tag_dict.keys():
            print('tag', ele,  'not exists in tag_dict, please modify')
            return 
    # the label after mapping
    Y = [tag_dict[ele[1]] for ele in pos_tags0]

    if len(X_wtv) != len(X_uie) or len(X_wtv) != len(Y):
        print("Not same length, X_wtv, X_uie, Y", len(X_wtv), len(X_uie), len(Y))

            
    return X_wtv, X_uie, Y, list(new_inverted_index.keys())

### 2.1.3. Save train and test data
use the data generated and the txt to construct train and test labels for NN

In [229]:
k = 10

uie_dat_name = 'GDM5_test800.npy'
txt_file_name = 'test800.txt'
X_wtv, X_uie, Y, words_list = retrive_X_Y_all(uie_dat_name, txt_file_name, k)

np.save('X_wtv.npy', X_wtv)
np.save('X_uie.npy', X_uie)
np.save('Y.npy', Y)
np.save('word_list.npy', words_list)

print(len(X_wtv), len(X_wtv), len(Y),len(words_list))


PCA total explained ratio: 0.17989720993887387
original occurance:  Counter({'NN': 237, 'JJ': 167, 'NNS': 97, 'RB': 85, 'VBD': 44, 'VBP': 44, 'VBG': 28, 'VBN': 25, 'VB': 17, 'VBZ': 17, 'IN': 13, 'JJS': 9, 'MD': 4, 'RBR': 3, 'CD': 3, 'DT': 2, 'PRP': 1, 'WP$': 1, 'JJR': 1, 'WDT': 1})
799 799 799 799


In [230]:

uie_dat_name = "GDM3_test1000.npy"
txt_file_name = "test1000.txt"
X_wtv, X_uie, Y, words_list= retrive_X_Y_all(uie_dat_name, txt_file_name, k)

np.save('X_wtv.npy', X_wtv)
np.save('X_uie.npy', X_uie)
np.save('Y.npy', Y)
np.save('word_list.npy', words_list)

print(len(X_wtv), len(X_wtv), len(Y),len(words_list))

PCA total explained ratio: 0.17488964472028978
original occurance:  Counter({'NN': 418, 'JJ': 252, 'NNS': 145, 'RB': 107, 'VBD': 69, 'VBP': 66, 'VBG': 44, 'VBN': 40, 'VBZ': 26, 'IN': 16, 'VB': 14, 'JJS': 9, 'MD': 6, 'JJR': 4, 'CD': 3, 'PRP': 2, 'DT': 2, 'RBR': 1, 'WP$': 1, 'WDT': 1, 'WRB': 1})
1227 1227 1227 1227


**Observation**: it is superising that when dimension is reduced to 10, the preserved information is approximatedly 18%; even if we use reduced dimension to 50, the presevered information is still 50


## 2.2. Universal isomorphic embedding (sentence as graph node)
- Use sentence as the graph nodes to construct vector 
- Use the vectors of the sentence to reconstruct word vector using a weighted recombination related to the vertex degree

In [198]:
def clean_sentences(sentences):
    tokens = []
    
    for sentence in sentences:
        token = word_tokenize(sentence)
        token = [w.lower() for w in token]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in token]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        tokens.append(words)

    return tokens

def get_vertices(sentences):
    graph_vertices = []
    inverse_words = []
    for sentence in sentences:
        sentence_set = set(sentence)
        vertex = {}
        inverse = {}
        for index, word in enumerate(sentence_set):
            vertex[word] = index
            inverse[index] = word
        graph_vertices.append(vertex)
        inverse_words.append(inverse)
    return graph_vertices, inverse_words

def get_words_set(graph_vertices):
    words_set = set()
    for i in range(len(graph_vertices)):
        for key in graph_vertices[i]:
            words_set.add(key)
    return words_set

def get_words_list(graph_vertices):
    words_list = []
    for i in range(len(graph_vertices)):
        words_set = set()
        for key in graph_vertices[i]:
            words_set.add(key)
        words_list.append(words_set)
    return words_list

def get_decoded_sentences(graph_vertices, sentences):
    decoded_sentences = []
    count = 0
    for vertex in graph_vertices:
        decoded_sentence = [vertex[word] for word in sentences[count]]
        decoded_sentences.append(decoded_sentence)
        count += 1
    return decoded_sentences

##############################
# Modified
##############################
def generate_edge(word_list, n):
    """
    use a sliding window of size n, count the number of pairs as edge weights
    input size: n
    return: nxn matrix
    """
    edge_mat = np.zeros([max(word_list)+1, max(word_list)+1])
    pair_list = []

    for i in range(len(word_list)-n+1):
        pair_list.append(list(permutations(word_list[i:i+n],2))) 

    for i in range(len(pair_list)):
        for it in pair_list[i]:
            edge_mat[it[0],it[1]] += 1
    return edge_mat # (a,b), (b,a) counted twice

def get_adj_mats(decoded_sentences, window_size):
    adj_mats = []
    for decoded_sentence in decoded_sentences:
        if len(decoded_sentence) >= 2:
            adj_mats.append(generate_edge(decoded_sentence ,window_size))
        else:
            adj_mats.append(None)
    return adj_mats

def get_edges(adj_mats):
    edges_sets = []
    for i in range(len(adj_mats)):
        edges = []
        for j in range(len(adj_mats[i])):
            for z in range(j):
                if adj_mats[i][j][z] != 0:
                    edges.append((j,z))
        edges_sets.append(edges)
    return edges_sets

def get_degree_contributions(adj_mats, edges_sets):
    degree_contributions = []
    for i in range(len(edges_sets)):
        degree = dict()
        for edge in edges_sets[i]:
            degree[edge[0]] = 0
            degree[edge[1]] = 0
        for edge in edges_sets[i]:
            degree[edge[0]] += 1
            degree[edge[1]] += 1
        vertices_set = set()
        for edge in edges_sets[i]:
            vertices_set.add(edge[0])
            vertices_set.add(edge[1])
        for vertex in vertices_set:
            degree[vertex] /= 2*len(edges_sets[i])
        degree_contributions.append(degree)
    return degree_contributions

def pad(a, MAX_len):
    """ 
    a: Array to be padded
    MAX_len: return shape MAX_len X MAX_len

    return result (zero padded matrix with shape=(MAX_len,MAX_len))
    """

    result = np.zeros((MAX_len,MAX_len))
    result[:a.shape[0],:a.shape[1]] = a
    # pad(a, [MAX_len,MAX_len], len(a)*len(a))
    return result

def get_word_vector(words_set, words_list, graph_vertices, X_proj, degree_contributions):    
    word_vector = dict()
    # each word
    for word in words_set:
        cur_vec = np.zeros(np.shape(X_proj[0]))
        sum = 0.0
        # iterate each sentence
        for i in range(len(words_list)):
            # if word in the the sentence 
            if word in words_list[i]:
                # print(i,word)
                if len(degree_contributions[i]) != 0:
                    cur_vec += degree_contributions[i][graph_vertices[i][word]]*X_proj[i]
                    sum += degree_contributions[i][graph_vertices[i][word]]
        if sum != 0.0:
            cur_vec /= sum
        word_vector[word] = cur_vec
    return word_vector


In [199]:
# read file
txt_file_name = 'test800.txt'
text_content = open(txt_file_name, encoding = 'utf-8', errors = 'ignore').read()
sentences = sent_tokenize(text_content)
sentences = clean_sentences(sentences)


# remove sentences with zero or one word 
cleaned_sentence = []
for ele in sentences:
    if not len(ele) <= 1:
        cleaned_sentence.append(ele)
    else: 
        # print(ele)
        pass
        
sentences = cleaned_sentence
graph_vertices, inverse_words = get_vertices(sentences)
words_list = get_words_list(graph_vertices)
words_set = get_words_set(graph_vertices)
decoded_sentences = get_decoded_sentences(graph_vertices, sentences)

adj_mats = get_adj_mats(decoded_sentences, 3)
edges_sets = get_edges(adj_mats)
degree_contributions = get_degree_contributions(adj_mats, edges_sets)

GDM_list = []
##################################################
# update non-connected vertices with shortest path
for i in range(len(adj_mats)):
    GDM = np.array(adj_mats[i])
    # GDM[GDM == 0] = 1e10
    # update non-connected vertices with shortest path
    if adj_mats[i] is not None:
        GDM[GDM == 0] = 1e30
        # dignal = 0
        for i in range(len(GDM)):
            GDM[i,i] = 0
        for z in range(len(GDM)):
            for u in range(len(GDM)):
                for v in range(len(GDM)): 
                    if GDM[u,v] > GDM[u,z] + GDM[z,v]:
                        GDM[u,v] = GDM[u,z] + GDM[z,v]   
    else:
        GDM = None

    GDM_list.append(GDM)

# zero padding s.t. all sentence vector has same shape 
MAX_len = 0
cleaned_GDM_list = []
clean_idx_list = []
for ele in GDM_list:
    if ele is None:
        clean_idx_list.append(False)
        pass
    else:
        clean_idx_list.append(False)
        cleaned_GDM_list.append(ele)
        if len(ele) > MAX_len:
            MAX_len = len(ele)

# list after zero padding 
GDM_pedded_list = []

for i in range(len(cleaned_GDM_list)):
    GDM_pedded_list.append(pad(cleaned_GDM_list[i], MAX_len))

print(len(GDM_pedded_list), len(graph_vertices), len(degree_contributions), len(clean_idx_list))

word_vector = get_word_vector(words_set, words_list, graph_vertices, GDM_pedded_list, degree_contributions)

107 107 107 107


- The word not in the constructed dict 

In [204]:
word_list = np.load('word_list.npy')
for key in word_vector.keys():
    word_vector[key] = list(word_vector[key].flat)
    
X_final = []
word_mising = []
for word in word_list:
    if word in word_vector.keys():
        X_final.append(word_vector[word])
    else:
         X_final.append(np.zeros(np.shape(word_vector['questions'])))
         word_mising.append(word)
         print(word)

cannot
self
counter
educate
monkeys
ninety
price
serving
co
powder
believe
high
born
balance
current
prices
reliance
operate
holders
cloth
houses
alms


In [213]:
# save file 
k = 10
pca = PCA(n_components=k)
X_uiesg = pca.fit_transform(X_final) # google vector 
print('PCA total explained ratio:', sum(pca.explained_variance_ratio_))

np.save('X_uiesg.npy',X_uiesg)

len(X_uiesg)

PCA total explained ratio: 0.9541151275871456


799

## Observation 
One possibility of a slight better performance as opposed one use words as node to construct word vector is that after PCA, the more information is presevered 95% vs. 18%. 
 


### 3.1.2. Kmeans for prediction

In [231]:
from sklearn.cluster import KMeans

# X = word_vec_dict[2,2]
n_clusters = 5
Y_pred_dict = {} # key: n,k ;stroe all predictions with sliding window length n, reduced dimenion k

Y_w2v_pred_dict = {}

# word2node
# # for n in N_RANGE:
# for k in K_RANGE:
X = X_uie
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
Y_uis_pred= kmeans.labels_

# word2vec
# for k in K_RANGE:
X = X_wtv
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
Y_w2v_pred = kmeans.labels_



In [ ]:
print(len(Y),len(Y_pred_dict[n,k]),len(X[:,]))

In [ ]:
label_set

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_pred_dim23(n, k, word_vec_dict, Y_pred_dict, Y_w2v_dict):
    """ plot the prediction of Kmeans 
    Input
        n: sliding window length
        k: dim reduced using PCA
        word_vec_dict: key = [n,k] value = X with dim=[len(word_set),k] 
        Y_pred_dict: key = [n,k] value = Y_pred with dim=[len(word_set),k] 
        
    Return 
        two plots
    """
    X =  word_vec_dict[n,k]

    n_clusters = max(Y)+1

    if k == 2:
        plt.figure(figsize = (15, 5))
        for cluster in range(n_clusters):
            idx = (np.array(Y) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], s=50, label = label_num_to_tag_dict[cluster])
        plt.title('Reality')
        plt.legend()
        plt.grid(1)

        plt.figure(figsize = (15, 5))
        for cluster in range(n_clusters):
            idx = (np.array(Y_w2v_pred_dict[k]) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], s=50, label = label_num_to_tag_dict[cluster])
        plt.title('word2vec')
        plt.legend()
        plt.grid(1)

        plt.figure(figsize = (15, 5))
        for cluster in range(n_clusters):
            idx = (np.array(Y_pred_dict[n,k]) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], s=50, label = label_num_to_tag_dict[cluster])
        plt.title('word2node')

        plt.legend()
        plt.grid(1)

    elif k == 3:
        fig1 = plt.figure(figsize = (15, 5))
        ax = Axes3D(fig1)
        for cluster in range(n_clusters):
            idx = (np.array(Y) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], X[idx, 2], marker = 'o', label = label_num_to_tag_dict[cluster])
        plt.title('Reality')
        plt.legend()
        plt.grid(1)

        fig2 = plt.figure(figsize = (15, 5))
        ax = Axes3D(fig2)
        for cluster in range(n_clusters):
            idx = (np.array(Y_w2v_pred_dict[k]) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], X[idx, 2], marker = 'o', label = label_num_to_tag_dict[cluster])
        plt.title('word2vec')
        plt.legend()
        plt.grid(1)

        fig3 = plt.figure(figsize = (15, 5))
        ax = Axes3D(fig3)
        for cluster in range(n_clusters):
            idx = (np.array(Y_pred_dict[n,k]) == cluster)
            plt.scatter(X[idx, 0], X[idx, 1], X[idx, 2], marker = 'o', label = label_num_to_tag_dict[cluster])
        plt.title('word2node')
        plt.legend()
        plt.grid(1)

    else:
        print('Cannot deal with k>=3')



            
    plt.show()



In [ ]:

n = 2
k = 2
plot_pred_dim23(n, k, word_vec_dict, Y_pred_dict, Y_w2v_dict)


In [ ]:

n = 2
k = 3
plot_pred_dim23(n, k, word_vec_dict, Y_pred_dict, Y_w2v_dict)


In [ ]:
word_vec_dict

## 3.2. Setiment classification

### 2.1. Graph Construction 


In [ ]:
edge_mat = edge_mat_dict[10]


words = text.split()
words = [word for word in words if word not in stopwords.words('english')]

In [ ]:
vertex_list

In [ ]:

G0 = nx.Graph()
G0.add_nodes_from(range(len(vertex_list)))

# flatten the matrix as [x_idx,y_idx, value]
XX,YY = np.meshgrid(np.arange(edge_mat.shape[1]),np.arange(edge_mat.shape[0]))
idx = [edge_mat.ravel() != 0]
flattened_mat = np.vstack((XX.ravel()[idx], YY.ravel()[idx], edge_mat.ravel()[idx])).T
flattened_mat = [tuple(ele) for ele in flattened_mat]


G0.add_weighted_edges_from(flattened_mat)


# nx.all_pairs_dijkstra(G0)


In [ ]:
>>> for (u, v, wt) in G0.edges.data('weight'):
...     if wt > 0.5: print('(%d, %d, %.3f)' % (u, v, wt))

In [ ]:
path = dict(nx.all_pairs_shortest_path(G0))


In [ ]:
path[11]

In [ ]:
GDM_new = np.zeros(np.shape(GDM_dict[10]))
for i in range(len(inverted_index.keys())):
    for j in range(len(inverted_index.keys())):
        GDM_new[i,j] = dict_ew[i][0][j]



In [ ]:
GDM_new

In [257]:
for source in range(len(inverted_index.keys())):
    if source % 100 == 0:
        print(source) 
    for target in range(source,len(inverted_index.keys())):
        if source != target:
            # shortest path from source to target 
            # print(nx.dijkstra_path_length(G0,source,target))
            # sp = nx.shortest_path_length(G0,source=source,target=target)
            # print(sp)
            G0.add_edge(source, target, weight=nx.dijkstra_path_length(G0,source,target))

0
100


In [258]:
G0

In [ ]:
nx.to_numpy_matrix(G0, nodelist=[range(len(inverted_index.keys()))])


In [ ]:
G = nx.Graph()
G.add_edge(1, 2, weight=3)
temp = G.get_edge_data(1,2)['weight']
temp += 5
G.add_edge(1, 2, weight= temp)  


In [ ]:
dict(nx.all_pairs_dijkstra(G))

list(inverted_index.keys())all_pairs_dijkstra

In [ ]:
plt.figure(figsize=(100,60))
plt.show()
nx.draw(G0)


In [ ]:
plt.figure(figsize=(100,60))
plt.show()

In [ ]:
G=nx.Graph()
G.add_edges_from([(1,2),(1,3)])
G.add_node("spam")      
nx.connected_components(G)

nx.draw(G)
nx.draw_random(G)
nx.draw_circular(G)
nx.draw_spectral(G)
plt.show()

In [ ]:
edge_mat